In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import log_loss
import numpy as np

In [ ]:
pred = []
file_pred=open('./data/querry_pred.csv', "r") # the stacknet prediction file
counter=0
for line in file_pred:
    splits=line.replace("\n","").split(",") 
    #print to the new file
    pred.append(float(splits[1]))
    counter+=1
    if counter%100000==0:
        print( " printing row %d " % (counter))
file_pred.close()


In [ ]:
nodl_train_fea = pd.read_csv('./data/train_no_dl_fea.csv')
nodl_train_fea1 = pd.read_csv('./data/train_no_dl_fea1.csv')
# dl_train_fea = pd.read_csv('./data/dl_train.csv')
# dl_train_fea1 = pd.read_csv('./data/dl_train_new.csv')

In [ ]:
dl_train_fea2 = pd.read_csv('./stack_dl_feature/dl_fea_train.csv')

In [ ]:
train_df = pd.read_csv('./data/train_org.csv',usecols=['is_duplicate'])

In [ ]:
col = [c for c in nodl_train_fea1.columns if ((c[:2]=='h_')or(c[:2]=='k_'))]+['w_porter_interaction',
 'w_porter_jaccard',
 'w_porter_levenshtein_1',
 'w_porter_levenshtein_2',
 'w_porter_sorensen']#
for c in col:
    nodl_train_fea[c] = nodl_train_fea1[c]
# for c in ['no_stops_dl', u'clean_dl', u'no_stops_birnn_dl',
#        u'clean_dl_birnn', u'no_stops_birnn_dl_difference',
#        u'clean_birnn_dl_difference', u'stemd_birnn',
#        u'no_stops_birnn_dl_glove', u'clean_difference_dl_glove',
#        u'clean_dl_birnn_glove', u'adj_n_v_nostopsdl_glove',
#        u'stemd_birnn_glove', u'stems_birnn_glove', u'stems_birnn_google']:#
#     nodl_train_fea[c] = dl_train_fea[c]
# nodl_train_fea['noun_verb_adj_birnn'] = dl_train_fea1.noun_verb_adj_birnn

In [ ]:
for c in dl_train_fea2.columns:
    nodl_train_fea[c] = dl_train_fea2[c]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(nodl_train_fea.drop([u'h_exactly_same',                                                         
                                                                           'x_difflib_sim',
                                                                           'DiceDistance_2gram',
 'magic_no_stops_q1',
 'magic_no_stops_q2',
 'magic_stems_no_stops_q1',
 'magic_stems_no_stops_q2',
 'magic_stems_q1',
 'magic_stems_q2',                                                                                                                        
                                                                            'IntersectRatio_3gramno_stops',
                                                                           u'CooccurrenceCount_2gramno_stops',
                                                                           u'CooccurrenceRatio_3gramno_stops',
                                                                           u'IntersectCount_2gramno_stops',
                                                                           u'DiceDistance_2gramno_stops',
                                                                           u'CooccurrenceCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           'DiceDistance_3gramno_stops',
                                                                           u'DiceDistance_3gram'],axis=1), train_df['is_duplicate'], test_size=0.1, random_state=0)

In [ ]:
x_train.columns

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'learning_rate': 0.05,
    'max_bin':255,
    'verbose':10,
'min_data_in_leaf':300}

# feature_name = [u'word_match', u'tfidf_word_match', u'dl_feature']
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000000,
                valid_sets=[lgb_train,lgb_eval],
                early_stopping_rounds=50,verbose_eval=50)

In [ ]:
import pandas as pd
test_nodl_fea = pd.read_csv('./data/test_no_dl_fea.csv')
test_nodl_fea1 = pd.read_csv('./data/test_no_dl_fea1.csv')
test_dl_fea = pd.read_csv('./data/dl_test.csv')
test_dl_fea1 = pd.read_csv('./data/dl_test_new.csv')

In [ ]:
col = [c for c in test_nodl_fea1.columns if ((c[:2]=='h_')or(c[:2]=='k_'))]+['w_porter_interaction',
 'w_porter_jaccard',
 'w_porter_levenshtein_1',
 'w_porter_levenshtein_2',
 'w_porter_sorensen']#
for c in col:
    test_nodl_fea[c] = test_nodl_fea1[c]
for c in [u'no_stops_dl', u'clean_dl', u'no_stops_birnn_dl',
       u'clean_dl_birnn', u'no_stops_birnn_dl_difference',
       u'clean_birnn_dl_difference', u'stemd_birnn',
       u'no_stops_birnn_dl_glove', u'clean_difference_dl_glove',
       u'clean_dl_birnn_glove', u'adj_n_v_nostopsdl_glove',
       u'stemd_birnn_glove']:#
    test_nodl_fea[c] = test_dl_fea[c]
test_nodl_fea['noun_verb_adj_birnn'] = test_dl_fea1.noun_verb_adj_birnn

In [ ]:
del test_nodl_fea1,test_dl_fea,test_dl_fea1

In [ ]:
test_fea = test_nodl_fea.drop([u'h_exactly_same',                                                         
                                                                           'x_difflib_sim',
                                                                           'DiceDistance_2gram',
 'magic_no_stops_q1',
 'magic_no_stops_q2',
 'magic_stems_no_stops_q1',
 'magic_stems_no_stops_q2',
 'magic_stems_q1',
 'magic_stems_q2',                                                                                                                        
                                                                            'IntersectRatio_3gramno_stops',
                                                                           u'CooccurrenceCount_2gramno_stops',
                                                                           u'CooccurrenceRatio_3gramno_stops',
                                                                           u'IntersectCount_2gramno_stops',
                                                                           u'DiceDistance_2gramno_stops',
                                                                           u'CooccurrenceCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           'DiceDistance_3gramno_stops',
                                                                           u'DiceDistance_3gram'],axis=1)

In [ ]:
del test_nodl_fea

In [ ]:
gbm.save_model('./ensem_model/model_6.txt')
print('Start predicting...')
# predict
y_pred = gbm.predict(test_fea, num_iteration=gbm.best_iteration)

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 50,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'learning_rate': 0.05,
    'max_bin':400}

# feature_name = [u'word_match', u'tfidf_word_match', u'dl_feature']
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=[lgb_eval],
                early_stopping_rounds=5)

In [ ]:
X= nodl_train_fea.drop([u'h_exactly_same',
                        'x_difflib_sim',
                        'DiceDistance_2gram',
 'magic_no_stops_q1',
 'magic_no_stops_q2',
 'magic_stems_no_stops_q1',
 'magic_stems_no_stops_q2',
 'magic_stems_q1',
 'magic_stems_q2',                                                                                                                        
                                                                            'IntersectRatio_3gramno_stops',
                                                                           u'CooccurrenceCount_2gramno_stops',
                                                                           u'CooccurrenceRatio_3gramno_stops',
                                                                           u'IntersectCount_2gramno_stops',
                                                                           u'DiceDistance_2gramno_stops',
                                                                           u'CooccurrenceCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           'DiceDistance_3gramno_stops',
                                                                           u'DiceDistance_3gram'],axis=1)

In [ ]:
len(X.columns)

In [ ]:
from scipy.sparse import csr_matrix,hstack
def fromsparsetofile(filename, array, deli1=" ", deli2=":",ytarget=None):    
    zsparse=csr_matrix(array)
    indptr = zsparse.indptr
    indices = zsparse.indices
    data = zsparse.data
    print(" data lenth %d" % (len(data)))
    print(" indices lenth %d" % (len(indices)))    
    print(" indptr lenth %d" % (len(indptr)))
    
    f=open(filename,"w")
    counter_row=0
    for b in range(0,len(indptr)-1):
        #if there is a target, print it else , print nothing
        if ytarget!=None:
             f.write(str(ytarget[b]) + deli1)     
             
        for k in range(indptr[b],indptr[b+1]):
            if (k==indptr[b]):
                if np.isnan(data[k]):
                    f.write("%d%s%f" % (indices[k],deli2,-1))
                else :
                    f.write("%d%s%f" % (indices[k],deli2,data[k]))                    
            else :
                if np.isnan(data[k]):
                     f.write("%s%d%s%f" % (deli1,indices[k],deli2,-1))  
                else :
                    f.write("%s%d%s%f" % (deli1,indices[k],deli2,data[k]))
        f.write("\n")
        counter_row+=1
        if counter_row%10000==0:    
            print(" row : %d " % (counter_row))    
    f.close()  
X=csr_matrix(X)
X =hstack([X]).tocsr()
fromsparsetofile("./data/train.sparse", X, deli1=" ", deli2=":",ytarget=train_df.is_duplicate.values) 

In [ ]:
import pandas as pd
test_nodl_fea = pd.read_csv('./data/test_no_dl_fea.csv')
test_nodl_fea1 = pd.read_csv('./data/test_no_dl_fea1.csv')
test_dl_fea = pd.read_csv('./data/dl_test.csv')
test_dl_fea1 = pd.read_csv('./data/dl_test_new.csv')

In [ ]:
col = [c for c in test_nodl_fea1.columns if ((c[:2]=='h_')or(c[:2]=='k_'))]+['w_porter_interaction',
 'w_porter_jaccard',
 'w_porter_levenshtein_1',
 'w_porter_levenshtein_2',
 'w_porter_sorensen']#
for c in col:
    test_nodl_fea[c] = test_nodl_fea1[c]
for c in [u'no_stops_dl', u'clean_dl',u'no_stops_birnn_dl','clean_dl_birnn','no_stops_birnn_dl_difference',
   u'clean_birnn_dl_difference','stemd_birnn','no_stops_birnn_dl_glove']:#
    test_nodl_fea[c] = test_dl_fea[c]
test_nodl_fea['noun_verb_adj_birnn'] = test_dl_fea1.noun_verb_adj_birnn

In [ ]:
del test_nodl_fea1,test_dl_fea,test_dl_fea1

In [ ]:
X= test_nodl_fea.drop([u'h_exactly_same',
                        'x_difflib_sim',
                        'DiceDistance_2gram',
 'magic_no_stops_q1',
 'magic_no_stops_q2',
 'magic_stems_no_stops_q1',
 'magic_stems_no_stops_q2',
 'magic_stems_q1',
 'magic_stems_q2',                                                                                                                        
                                                                            'IntersectRatio_3gramno_stops',
                                                                           u'CooccurrenceCount_2gramno_stops',
                                                                           u'CooccurrenceRatio_3gramno_stops',
                                                                           u'IntersectCount_2gramno_stops',
                                                                           u'DiceDistance_2gramno_stops',
                                                                           u'CooccurrenceCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           'DiceDistance_3gramno_stops',
                                                                           u'DiceDistance_3gram'],axis=1)

In [ ]:
del test_nodl_fea

In [ ]:
import numpy as np
np.shape(X)

In [ ]:
from scipy.sparse import csr_matrix,hstack
def fromsparsetofile(filename, array, deli1=" ", deli2=":",ytarget=None):    
    zsparse=csr_matrix(array)
    indptr = zsparse.indptr
    indices = zsparse.indices
    data = zsparse.data
    print(" data lenth %d" % (len(data)))
    print(" indices lenth %d" % (len(indices)))    
    print(" indptr lenth %d" % (len(indptr)))
    
    f=open(filename,"w")
    counter_row=0
    for b in range(0,len(indptr)-1):
        #if there is a target, print it else , print nothing
        if ytarget!=None:
             f.write(str(ytarget[b]) + deli1)     
             
        for k in range(indptr[b],indptr[b+1]):
            if (k==indptr[b]):
                if np.isnan(data[k]):
                    f.write("%d%s%f" % (indices[k],deli2,-1))
                else :
                    f.write("%d%s%f" % (indices[k],deli2,data[k]))                    
            else :
                if np.isnan(data[k]):
                     f.write("%s%d%s%f" % (deli1,indices[k],deli2,-1))  
                else :
                    f.write("%s%d%s%f" % (deli1,indices[k],deli2,data[k]))
        f.write("\n")
        counter_row+=1
        if counter_row%10000==0:    
            print(" row : %d " % (counter_row))    
    f.close()  
X=csr_matrix(X)
X =hstack([X]).tocsr()
fromsparsetofile("./data/test.sparse", X, deli1=" ", deli2=":",ytarget=None)

In [ ]:
fromsparsetofile("./data/test.sparse", X, deli1=" ", deli2=":",ytarget=None)

In [ ]:
del X

In [ ]:
nodl_train_fea = pd.read_csv('./data/train_no_dl_fea.csv')
#nodl_train_fea1 = pd.read_csv('./data/train_no_dl_fea1.csv')

In [ ]:
dl_train_fea = pd.read_csv('./data/dl_train.csv')

In [ ]:
dl_train_fea.columns

In [ ]:
train_df = pd.read_csv('./data/train_org.csv')

In [ ]:
#log_loss(train_df.is_duplicate.values,dl_train_fea.clean_dl.values)

In [ ]:
nodl_train_fea.columns

In [ ]:
dl_train_fea.columns

In [ ]:
#nodl_train_fea['is_duplicate'] = train_df.is_duplicate


In [ ]:
for c in nodl_train_fea1.columns:#
    nodl_train_fea[c] = nodl_train_fea1[c]
for c in ['clean_birnn_dl_difference',u'no_stops_dl', u'clean_dl',u'no_stops_birnn_dl','clean_dl_birnn', u'no_stops_birnn_dl_difference']:#
    nodl_train_fea[c] = dl_train_fea[c]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(nodl_train_fea.drop(['IntersectCount_3gram',
                                                                           'IntersectRatio_3gramno_stops',
                                                                           u'CooccurrenceCount_2gramno_stops',
                                                                           u'CooccurrenceRatio_3gramno_stops',
                                                                           u'IntersectCount_2gramno_stops',
                                                                           u'DiceDistance_2gramno_stops',
                                                                           u'CooccurrenceCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           u'IntersectCount_3gramno_stops',
                                                                           'DiceDistance_3gramno_stops',
                                                                           u'DiceDistance_3gram'],axis=1), train_df['is_duplicate'], test_size=0.1, random_state=0)

In [ ]:
kk = {}
for name,num in zip(gbm.feature_name(),list(gbm.feature_importance())):
    kk[name] = num
#print('Save model...')
print('Feature importances:', )
sorted(kk.iteritems(), key=lambda d:d[1], reverse = False)

In [ ]:
dl_test_fea = pd.read_csv('./data/dl_train.csv')

In [ ]:
for c in nodl_test_fea1.columns:
    nodl_test_fea[c] = nodl_test_fea1[c]
for c in ['clean_birnn_dl_difference',u'no_stops_dl', u'clean_dl',u'no_stops_birnn_dl','clean_dl_birnn', u'no_stops_birnn_dl_difference']:#
    nodl_test_fea[c] = dl_test_fea[c]

In [ ]:
def train_model(trainx,valx,trainy,valy,num_c):
    lgb_train = lgb.Dataset(trainx, trainy)
    lgb_eval = lgb.Dataset(valx.iloc[0:5000], valy.iloc[0:5000], reference=lgb_train)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': 25,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'learning_rate': 0.05,
        'max_bin':400}

    # feature_name = [u'word_match', u'tfidf_word_match', u'dl_feature']
    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    valid_sets=[lgb_eval],
                    early_stopping_rounds=5)
    print('Save model...')
    gbm.save_model('./ensem_model/model_'+str(num_c)+'.txt')

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)
train_test_split = []
train_ids = []
for train_index, test_index in skf.split(train_df.id, train_df.is_duplicate):
    train_test_split.append([train_index,test_index])
    train_ids.extend(test_index)

In [ ]:
from sklearn.metrics import log_loss
train_feature = []
test_feature = []
for num_c,spl in enumerate(train_test_split):
    train_x,val_x = nodl_train_fea.iloc[spl[0]],nodl_train_fea.iloc[spl[1]]
    train_y,val_y = train_df.is_duplicate.iloc[spl[0]],train_df.is_duplicate.iloc[spl[1]]
    best_model_epo = train_model(train_x,val_x,train_y,val_y,num_c)
    #train_feature.extend(train_pred)

In [ ]:
def test_model(path,val_x):
    bst = lgb.Booster(model_file=path)
    # can only predict with the best iteration (or the saving iteration)
    y_pred = bst.predict(val_x)
    return y_pred

In [ ]:
train_feature = []
test_feature = []
for num_c,spl in enumerate(train_test_split):
    path = './ensem_model/model_'+str(num_c)+'.txt'
    val_x = nodl_train_fea.iloc[spl[1]]
    train_pred = test_model(path,val_x)
    train_feature.extend(train_pred)
    #test_feature.append(test_pred)

In [ ]:
train_dl = np.zeros(len(train_ids))
for i,id_ in enumerate(train_ids):
    train_dl[id_] = train_feature[i]

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'learning_rate': 0.05,
    'max_bin':400}

# feature_name = [u'word_match', u'tfidf_word_match', u'dl_feature']
print('Start training...')
# train
gbm1 = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=[lgb_eval],
                early_stopping_rounds=5)

print('Save model...')

In [ ]:
gbm.save_model('./ensem_model/model_2.txt')

print('Start predicting...')
# predict
y_pred = gbm.predict(test_nodl_fea, num_iteration=gbm.best_iteration)

In [ ]:
len(y_pred)

In [ ]:
df_test_o = pd.read_csv('test_final.csv')

In [ ]:
# a = 0.165 / 0.37    
# b = (1 - 0.165) / (1 - 0.37)
# final_pred = map(lambda x: (a*x/(a*x + b*(1-x))), y_pred)

In [ ]:
a = 0.165 / 0.37    
b = (1 - 0.165) / (1 - 0.37)

In [ ]:
x_test_dl = np.zeros_like(test_dl_featuers["model_0"].values)
for i in range(len(ensem_files)):
    x_test_dl = x_test_dl + test_dl_featuers["model_"+str(i)].apply(lambda x: (a*x/(a*x + b*(1-x))))*weights[i]

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'learning_rate': 0.05,
    'max_bin':400}

# feature_name = [u'word_match', u'tfidf_word_match', u'dl_feature']
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=[lgb_eval],
                early_stopping_rounds=15)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

# print('Start predicting...')
# # predict
# y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
# # eval
# print('The rmse of prediction is:', log_loss(y_test, y_pred))

# print('Dump model to JSON...')
# # dump model to json (and save to file)
# model_json = gbm.dump_model()

# with open('model.json', 'w+') as f:
#     json.dump(model_json, f, indent=4)


print('Feature names:', gbm.feature_name())

print('Calculate feature importances...')
# feature importances
print('Feature importances:', list(gbm.feature_importance()))

In [ ]:
pos_train = x_train[y_train == 1]
neg_train = x_train[y_train == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train
